# Assignment 6: Urban Street Networks and Interactive Web Maps

## Myron Bañez

## Part 1: Visualizing crash data in Philadelphia

### 1.1 Load the geometry for the region being analyzed

We'll analyze crashes in the "Central" planning district in Philadelphia, a rough approximation for Center City. [Planning districts](https://www.opendataphilly.org/dataset/planning-districts) can be loaded from Open Data Philly. Read the data into a GeoDataFrame using the following link:

http://data.phl.opendata.arcgis.com/datasets/0960ea0f38f44146bb562f2b212075aa_0.geojson

Select the "Central" district and extract the geometry polygon for only this district. After this part, you should have a polygon variable of type `shapely.geometry.polygon.Polygon`.

In [ ]:
import osmnx as ox
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import altair as alt

In [ ]:
url = "http://data.phl.opendata.arcgis.com/datasets/0960ea0f38f44146bb562f2b212075aa_0.geojson"
hoods = gpd.read_file(url)
sel = hoods['DIST_NAME'].isin(['Central'])
central = hoods.loc[sel]
central

In [ ]:
ax = ox.project_gdf(central).plot(fc="lightblue", ec="gray")
ax.set_axis_off()

central_outline = central.geometry.unary_union
central_outline

type(central_outline)

### 1.2 Get the street network graph

Use OSMnx to create a network graph (of type 'drive') from your polygon boundary in 1.1.

In [ ]:
G_central = ox.graph_from_polygon(central_outline, network_type='drive')

ox.plot_graph(ox.project_graph(G_central), node_size=0);

### 1.3 Convert your network graph edges to a GeoDataFrame

Use OSMnx to create a GeoDataFrame of the network edges in the graph object from part 1.2. The GeoDataFrame should contain the edges but not the nodes from the network.

In [ ]:
type(G_central)

central_edges = ox.graph_to_gdfs(G_central, 
                                edges=True, 
                                nodes=False)

ax = central_edges.to_crs(epsg=3857).plot(color='gray')

# add the neighborhood boundaries
boundary = gpd.GeoSeries([central_outline], crs='EPSG:4326')
boundary.to_crs(epsg=3857).plot(ax=ax, facecolor='none', edgecolor='red', linewidth=3, zorder=2)

ax.set_axis_off()

### 1.4 Load PennDOT crash data

Data for 2021 crashes (of all types) is available at the following path:

`./data/CRASH_PHILADELPHIA_2021.csv`

The data was downloaded for Philadelphia County [from here](https://crashinfo.penndot.gov/PCIT/welcome.html).

In [ ]:
penndot = pd.read_csv("./data/CRASH_PHILADELPHIA_2021.csv")


### 1.5 Convert the crash data to a GeoDataFrame

You will need to use the `DEC_LAT` and `DEC_LONG` columns for latitude and longitude.

The full data dictionary for the data is [available here](http://pennshare.maps.arcgis.com/sharing/rest/content/items/ffe20c6c3c594389b275c6772a281bcd/data)

In [ ]:
penndot['geometry'] = gpd.points_from_xy(penndot['DEC_LONG'], penndot['DEC_LAT'])
penndot = gpd.GeoDataFrame(penndot, geometry='geometry', crs="EPSG:4326")
penndot

### 1.6 Trim the crash data to Center City

1. Get the boundary of the edges data frame (from part 1.3). Accessing the `.geometry.unary_union.convex_hull` property will give you a nice outer boundary region.
1. Trim the crashes using the `within()` function of the crash GeoDataFrame to find which crashes are within the boundary.

There should be about 1,600 crashes within the Central district.

In [ ]:
central_edges_1 = central_edges.geometry.unary_union.convex_hull
penndot1 = penndot.within(central_edges_1)
penndot2 = penndot.loc[penndot1]
penndot2

### 1.7 Re-project our data into an approriate CRS

We'll need to find the nearest edge (street) in our graph for each crash. To do this, `osmnx` will calculate the distance from each crash to the graph edges. For this calculation to be accurate, we need to convert from latitude/longitude 

**We'll convert the local state plane CRS for Philadelphia, EPSG=2272**

#### Two steps:
1. Project the graph object (`G`) using the `ox.project_graph`. Run `ox.project_graph?` to see the documentation for how to convert to a specific CRS. 
1. Project the crash data using the `.to_crs()` function.

In [ ]:
G_projected = ox.project_graph(G_central, to_crs=2272)
ox.plot_graph(G_projected);

penndot2 = penndot.to_crs(epsg=2272)
penndot2

### 1.8 Find the nearest edge for each crash

See: `ox.distance.nearest_edges()`. It takes three arguments:

- the network graph
- the longitude of your crash data (the `x` attribute of the `geometry` column)
- the latitude of your crash data (the `y` attribute of the `geometry` column)

You will get a numpy array with 3 columns that represent `(u, v, key)` where each `u` and `v` are the node IDs that the edge links together. We will ignore the `key` value for our analysis.

In [ ]:
crash_x = penndot2.geometry.x.dropna()
crash_x

In [ ]:
crash_y = penndot2.geometry.y.dropna()
crash_y

In [ ]:
central_near_edge = ox.distance.nearest_edges(G_projected,  crash_x, crash_y) 
central_near_edge

### 1.9 Calculate the total number of crashes per street

1. Make a DataFrame from your data from part 1.7 with three columns, `u`, `v`, and `key` (we will only use the `u` and `v` columns)
1. Group by `u` and `v` and calculate the size
1. Reset the index and name your `size()` column as `crash_count`

After this step you should have a DataFrame with three columns: `u`, `v`, and `crash_count`.

In [ ]:
central_near_edge1 = pd.DataFrame(central_near_edge, columns = ['u','v', 'key'])
central_near_edge1

central_near_edge2 = central_near_edge1.groupby(['u', 'v'])['key'].size()
central_near_edge2 = central_near_edge2.reset_index()
central_near_edge2

central_near_edge2.rename(columns = {'key': 'crash_count'}, inplace = True)
central_near_edge2

### 1.10 Merge your edges GeoDataFrame and crash count DataFrame

You can use pandas to merge them on the `u` and `v` columns. This will associate the total crash count with each edge in the street network. 

**Tips:** 
   - Use a `left` merge where the first argument of the merge is the edges GeoDataFrame. This ensures no edges are removed during the merge.
   - Use the `fillna(0)` function to fill in missing crash count values with zero.

In [ ]:
edges_data = ox.graph_to_gdfs(G_projected, 
                                edges=True, 
                                nodes=False)

edges_data


final_data = edges_data.merge(central_near_edge2, left_on='u', right_on='v')
final_data = final_data.fillna(0)
final_data

### 1.11 Calculate a "Crash Index"

Let's calculate a "crash index" that provides a normalized measure of the crash frequency per street. To do this, we'll need to:

1. Calculate the total crash count divided by the street length, using the `length` column
1. Perform a log transformation of the crash/length variable — use numpy's `log10()` function
1. Normalize the index from 0 to 1 (see the lecture notes for an example of this transformation)

**Note: since the crash index involves a log transformation, you should only calculate the index for streets where the crash count is greater than zero**.

After this step, you should have a new column in the data frame from 1.9 that includes a column called part 1.9.

In [ ]:
final_data["crash_index"] = (final_data["crash_count"] / final_data["length"])
final_data["crash_index_log"] = np.log10(final_data['crash_index'])
final_data

# Minimum
min_val = final_data['crash_index_log'].min()
# Maximum
max_val = final_data['crash_index_log'].max()
# Calculate a normalized column
normalized = (final_data['crash_index_log'] - min_val) / (max_val - min_val)
# Add to the dataframe
final_data['part 1.9'] = normalized

final_data

### 1.12 Plot a histogram of the crash index values

Use matplotlib's `hist()` function to plot the crash index values from the previous step.

You should see that the index values are Gaussian-distributed, providing justification for why we log-transformed!

In [ ]:
hist_data = final_data['part 1.9']
plt.hist(hist_data)
plt.show() 

### 1.13 Plot the street networks, colored by the crash index

You can use GeoPandas to make the plot, coloring the streets by the crash index column.

**Tip:** if you use the viridis color map, try setting the facecolor of the axes as black for better constrast of the colors.

In [ ]:
ax.set_facecolor("black")

final_data.plot(column='part 1.9', cmap='magma_r',legend=True)

### 1.14 An interactive map of the crash index

In this part, we'll use Folium to make an interactive version of the map from the previous section. In this part, you will need to:

1. Initialize a Folium map centered on Philadelphia. The "Cartodb dark_matter" will be best if you want to use the viridis color map.
1. Add the street edges polygons to the map using the `Folium.GeoJson()` function.
1. Use a style function that applies a color to the edge geometries based on the value of the "crash index" column. 
    - See the crash index is defined from 0 to 1, you can pass this directly to a color map object, as in lecture.
    - You will need to convert the RGB color returned by the color map to a hex string
    - You can also set the "weight" attribute to change the width of the streets.
1. Add a GeoJsonTooltip object that includes the street name and crash index value so you can quickly identify which streets have the highest index values.

**Note:** if the Folium map is not rendering in the notebook, try removing unused columns — you should really only need the "geometry", "crash_index", and "name" columns.

In [ ]:
import folium
import matplotlib.colors as mcolors

cmap = plt.get_cmap('RdPu')
cmap(0)
mcolors.rgb2hex(cmap(0.0))
mcolors.rgb2hex(cmap(1.0))

def get_style(feature):
    """
    Given an input GeoJSON feature, return a style dict.
    
    Notes
    -----
    The color in the style dict is determined by the 
    "percent_no_internet_normalized" column in the 
    input "feature".
    """
    # Get the data value from the feature
    value = feature['properties']['part 1.9']
    
    # Evaluate the color map
    # NOTE: value must between 0 and 1
    rgb_color = cmap(value) # this is an RGB tuple
    
    # Convert to hex string
    color = mcolors.rgb2hex(rgb_color)
    
    # Return the style dictionary
    return {'weight': 0.5, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

final_data2 = final_data.drop(['osmid', 'oneway', 'highway', 'reversed', 'length', 'maxspeed', 'tunnel', 'u', 'v'], axis=1)
final_data2 = final_data2.drop(['lanes', 'bridge', 'ref', 'width', 'service', 'access', 'junction', 'crash_index', 'crash_index_log', 'normalized'], axis=1)
final_data2

In [ ]:
m = folium.Map(
    location=[39.99, -75.13],
    zoom_start=11,
    tiles = 'Cartodb dark_matter'
)

m

In [ ]:
folium.GeoJson(
    final_data2.to_crs(epsg=4326), 
    name='Philadelphia ZIP_codes',
    style_function=get_style,
    highlight_function=get_highlighted_style,
    tooltip=folium.GeoJsonTooltip(['name','part 1.9'])
).add_to(m)


In [ ]:
m

## Part 2: Interactive web maps with Folium

In this part, you'll visualize a geospatial data set, queried using an API, using Folium in Python. The choice of data set is up to you, but must satisfy a few requirements:

- The data must be pulled using an API
- The data should be in GeoJSON format and be Point features (latitude, longitude)
- Use Folium to create a map showing a heat map of the data, using the Leaflet.heat plugin

#### Notes

- See [lecture 10A](https://github.com/MUSA-550-Fall-2022/week-10/blob/main/lecture-10A.ipynb) for example maps using the data set of Philadelphia building permits. **Note: you must choose a data set other than the building permits data set that we used in class**.
- There are several options for data on OpenDataPhilly — any data set hosted on the CARTO SQL database with associated API documentation will work, similar to the shootings data set (you can use the `carto2gpd` library to do the querying).
- You can also choose a different API to use, as long as it satisfies the above requirements. To query the API, you can either use the `requests` module in Python or use `geopandas` directly to read the geojson returned by the API. For example:
  - the [Philadelphia bike share Indego](https://www.rideindego.com/about/data/) has an API of live station data in GeoJSON format: https://www.rideindego.com/stations/json/
  - Data.gov maintains a list of APIs in GeoJSON format: https://catalog.data.gov/dataset?res_format=GeoJSON. Note that not all of these are Point features


In [ ]:
import carto2gpd
url = "https://phl.carto.com/api/v2/sql"
table_name = "incidents_part1_part2"

where = f"dispatch_date LIKE '2020-%'"
where += "and text_general_code in ('Thefts')"
where += "and dc_dist in ('16')"

crime = carto2gpd.get(url, table_name, where=where)
crime

In [ ]:
crime1 = crime.dropna()

crime_coords = crime1[['point_y', 'point_x']].values
crime_coords

In [ ]:
from folium.plugins import HeatMap


# Initialize map
m = folium.Map(
    location=[39.99, -75.13],
    tiles='Cartodb Positron',
    zoom_start=12
)


# Add heat map coordinates
HeatMap(crime_coords).add_to(m)

m